In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Get Data
g=open('sentiment-network/reviews.txt','r')
text=g.read()

In [3]:
#Remove punctuation
from string import punctuation
text=[eachchar for eachchar in text if eachchar not in punctuation]
text=''.join(text)

In [4]:
allwords=text.replace('\n',' ')
reviews=text.split('\n')
reviews=[review.split(' ') for review in reviews]

In [5]:
# Convert words to integers
def word_to_int(text):
    uniqwords=set(text.split(' '))
    vocab_to_int={word:index for index,word in enumerate(uniqwords)}
    int_to_vocab={index:word for index,word in enumerate(uniqwords)}
    return vocab_to_int,int_to_vocab
vocab_to_int,int_to_vocab=word_to_int(allwords)

In [6]:
reviews_ints=[]
for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])
    
reviews_ints=np.array([review[:50] for review in reviews_ints if len(review) >1 ])

In [7]:
labeldata=open('sentiment-network/labels.txt','r')

In [8]:
labeldata=labeldata.read()
labeldata=labeldata.split('\n')
labeldata=[1 if label=='positive' else 0 for label in labeldata]
labeldata=np.array(labeldata[:25000])


In [9]:
def get_batches(x,y,batchsize=100):
    num_batches=len(x)//batchsize
    x=x[:batchsize*num_batches]
    y=y[:batchsize*num_batches]
    for i in range(0,len(x),batchsize):
        yield x[i:i+batchsize],y[i:i+batchsize]

In [10]:
# Hyperparameters
batch_size=100
lstm_size=256
lstm_layers=1
n_words=len(vocab_to_int)
embed_size=300
learning_rate=0.1
epochs=1

In [11]:
# Build Graph
graph=tf.Graph()
with graph.as_default():
    #REVIEW why shape is None,None
    inputs=tf.placeholder(tf.int32,shape=[None,None],name='inputs')
    labels=tf.placeholder(tf.int32,shape=[None,None],name='labels')
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
#Embedding
    embed_size=300
    embedding=tf.Variable(tf.random_uniform((n_words,embed_size),-1,1))
    embed=tf.nn.embedding_lookup(embedding, inputs)
    
#LSTM cell
    onecell=tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop=tf.contrib.rnn.DropoutWrapper(onecell,output_keep_prob=keep_prob)
    cell=tf.contrib.rnn.MultiRNNCell([drop]*lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    
#Pass embedded to LSTM cell
    outputs,final_state=tf.nn.dynamic_rnn(cell=cell,inputs=embed,initial_state=initial_state)
    predictions=tf.contrib.layers.fully_connected(outputs[:,-1],1,activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [12]:
#Training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1,epochs+1):
        state = sess.run(initial_state)
        print("Epoch - ",i)
        batch=get_batches(reviews_ints,labeldata)
        
        for batchnumber in range(1,(len(reviews_ints)//batch_size)+1):
            print("Batch - ",batchnumber)
            x,y=next(batch)
            feed_dict={inputs:x,labels:y[:, None],keep_prob: 0.5,initial_state: state}
            loss,_=sess.run([cost, optimizer],feed_dict=feed_dict)
            print("Batch - ",batchnumber,"   Loss", loss)

Epoch -  1
Batch -  1


ValueError: setting an array element with a sequence.